<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week9_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 9 Practical (Python, Colab)
**Topic:** Concurrency & parallelism — threads/locks, actors/channels; race‑conditions & deadlocks
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This PH demonstrates shared‑memory concurrency pitfalls (lost updates, deadlocks) and a message‑passing alternative (actors/channels). Fits Week‑9 of the outline.

### Learning goals (≈60 minutes)
- Observe a **race condition** and then fix it with a **mutex** (mutual exclusion).
- Create and diagnose a **deadlock**, then refactor to a **lock ordering** discipline.
- Build a tiny **actor/channel** pipeline (no shared mutable state).

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check (Python 3.10+)
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — threads and queues available.")

In [ ]:
# =====================================
# Part 1 — Race condition: lost updates
# =====================================
import threading, time, random

counter = 0                                    # Shared state (global integer)
N_THREADS = 8                                  # Number of threads to run
N_INCREMENTS = 5000                            # Each thread increments this many times

def racy_inc():
    global counter
    # Non-atomic read-modify-write with deliberate yield to magnify the race
    tmp = counter
    time.sleep(0)                              # Yield to scheduler (encourage interleaving)
    counter = tmp + 1

def worker_racy():
    for _ in range(N_INCREMENTS):
        racy_inc()

# Run the racy experiment
counter = 0
threads = [threading.Thread(target=worker_racy) for _ in range(N_THREADS)]
for t in threads: t.start()
for t in threads: t.join()

expected = N_THREADS * N_INCREMENTS
print("RACY  - expected:", expected, "| got:", counter, "| lost:", expected - counter)
assert counter < expected, "If this rarely equals expected, rerun to observe the race."

In [ ]:
# =====================================
# Part 1b — Fix with a mutex
# =====================================
lock = threading.Lock()

def safe_inc():
    global counter
    with lock:                                 # Mutual exclusion for the critical section
        tmp = counter
        counter = tmp + 1

def worker_safe():
    for _ in range(N_INCREMENTS):
        safe_inc()

counter = 0
threads = [threading.Thread(target=worker_safe) for _ in range(N_THREADS)]
for t in threads: t.start()
for t in threads: t.join()

expected = N_THREADS * N_INCREMENTS
print("SAFE  - expected:", expected, "| got:", counter)
assert counter == expected
print("ok  - mutex prevents lost updates")

In [ ]:
# =====================================
# Part 2 — Deadlock demo and fix
# =====================================
A = threading.Lock()
B = threading.Lock()

def t1():
    with A:                                    # Acquire A then B
        time.sleep(0.001)
        B.acquire()
        try:
            time.sleep(0.001)                  # Simulate work
        finally:
            B.release()

def t2():
    with B:                                    # Acquire B then A (reverse order!)
        time.sleep(0.001)
        A.acquire()
        try:
            time.sleep(0.001)
        finally:
            A.release()

# Launch threads and detect potential deadlock by timeout
th1 = threading.Thread(target=t1, daemon=True)
th2 = threading.Thread(target=t2, daemon=True)
th1.start(); th2.start()
th1.join(timeout=0.2); th2.join(timeout=0.2)

if th1.is_alive() and th2.is_alive():
    print("DEADLOCK detected: both threads waiting (different lock order).")
else:
    print("No deadlock this run (timing-dependent); rerun the cell if needed.")

# Fix: impose a global lock ordering (always acquire A then B)
def t1_fix():
    with A:
        with B:
            time.sleep(0.001)

def t2_fix():
    with A:                                    # same order: A then B
        with B:
            time.sleep(0.001)

th1 = threading.Thread(target=t1_fix)
th2 = threading.Thread(target=t2_fix)
th1.start(); th2.start()
th1.join(); th2.join()
print("ok  - fixed by consistent lock ordering (no deadlock)")

In [ ]:
# =====================================
# Part 3 — Actors via Queues (message passing)
# =====================================
from queue import Queue, Empty

class Actor(threading.Thread):
    def __init__(self, name, handler):
        super().__init__(daemon=True)
        self.name = name
        self.mailbox = Queue()
        self.handler = handler
        self._stop = False

    def send(self, msg):
        self.mailbox.put(msg)

    def run(self):
        while not self._stop:
            try:
                msg = self.mailbox.get(timeout=0.1)
            except Empty:
                continue
            if msg == ("STOP",):
                self._stop = True
                continue
            self.handler(self, msg)

# Example: Counter actor (encapsulates state)
def counter_handler(self, msg):
    # state kept as an attribute; no data races since only this thread mutates it
    if not hasattr(self, "count"):
        self.count = 0
    tag, *rest = msg
    if tag == "ADD":
        (n,) = rest
        self.count += n
    elif tag == "GET":
        (reply_q,) = rest
        reply_q.put(self.count)

counter = Actor("counter", counter_handler); counter.start()

# Example: Printer actor
def printer_handler(self, msg):
    tag, (text,) = msg[0], msg[1:] if len(msg)>1 else [("","")]
    if tag == "PRINT":
        print("[printer]", text[0])

printer = Actor("printer", printer_handler); printer.start()

# Use the actors
for i in range(10):
    counter.send(("ADD", i))
printer.send(("PRINT", f"Sent 10 ADD messages to counter."))

reply = Queue()
counter.send(("GET", reply))
val = reply.get(timeout=1.0)
print("counter value =", val)

# Shutdown
counter.send(("STOP",)); printer.send(("STOP",))
print("ok  - actor pipeline ran without shared-state races")

In [ ]:
# =====================================
# Part 4 — I/O-bound concurrency: throughput demo
# =====================================
def io_task(n):
    time.sleep(0.02)           # simulate I/O wait (e.g., network/disk)
    return n*n

def run_sequential(N=20):
    t0 = time.time()
    out = [io_task(i) for i in range(N)]
    return out, time.time() - t0

def run_threaded(N=20, T=10):
    t0 = time.time()
    out = [None]*N
    def worker(i):
        out[i] = io_task(i)
    threads = [threading.Thread(target=worker, args=(i,)) for i in range(N)]
    for t in threads: t.start()
    for t in threads: t.join()
    return out, time.time() - t0

seq_out, t_seq = run_sequential(30)
thr_out, t_thr = run_threaded(30)
print(f"sequential: {t_seq:.3f}s; threaded: {t_thr:.3f}s; same results? {seq_out==thr_out}")
print("ok  - threads improve I/O-bound throughput (GIL still serializes CPU-bound work)")

### 🧪 Your Turn (10–15 minutes)
1) Replace the actor `Queue()` with a **bounded** queue (e.g., `Queue(maxsize=2)`) to observe **backpressure**; add log prints when senders block.  
2) Modify the deadlock demo to use **`RLock`** and explain why it does (or does not) help.  
3) Create a **worker pool** actor that processes jobs sent to a channel and returns results via reply queues.

### ✍️ Reflection (2–3 sentences)
- Why does a mutex fix lost updates in Part 1, and why doesn’t it fix deadlocks in Part 2?  
- In what kinds of tasks do **threads** improve performance in CPython, and why?

In [ ]:
# Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
  "student_name": STUDENT_NAME,
  "student_id": STUDENT_ID,
  "timestamp": stamp,
  "checks": ["race-fixed", "deadlock-fixed", "actor-ok", "io-throughput"],
  "reflection": "(fill in here)"
}
with open("week9_submission.json", "w") as f:
  json.dump(submission, f, indent=2)
print("Saved week9_submission.json — upload with your notebook.")